In the notebook `LSS_AGN-01_preparation` we prepared a simulated data cube of an AGN for "observation" with Scopesim/METIS. If you do not have the files `AGN_sim_prepared.fits` and `AGN_sim_rotated_50.fits`, please run the first notebook before proceeding.

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from astropy.io import fits
%matplotlib inline

In [ ]:
import scopesim as sim
sim.bug_report()

# Edit this path if you have a custom install directory, otherwise comment it out. [For ReadTheDocs only]
sim.link_irdb("../../../")

If you haven't got the instrument packages yet, uncomment the following cell.

In [ ]:
# sim.download_packages(["METIS", "ELT", "Armazones"])

In [ ]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=['lss_n'],
                      properties={"!OBS.exptime": 3600, "!OBS.dit": None, "!OBS.ndit": None,
                                  "!SIM.spectral.spectral_bin_width": 3.0e-3})
metis = sim.OpticalTrain(cmd)

Create two `Source` objects from the files. The first one is with the original field orientation, the second one has the field rotated by 50 degrees. We also define a `Source` that represents blank sky for background subtraction.

In [ ]:
cube_01 = "AGN_sim_prepared.fits"
cube_02 = "AGN_sim_rotated_50.fits"

with fits.open(cube_01) as cube_hdul:
    src_01 = sim.Source(cube=cube_hdul)
with fits.open(cube_02) as cube_hdul:
    src_02 = sim.Source(cube=cube_hdul)

sky = sim.source.source_templates.empty_sky()

We observe each of the two sources and the blank sky; this takes a few minutes.

In [ ]:
metis.observe(src_01, update=True)
hdul_01 = metis.readout(detector_readout_mode="auto")[0]

In [ ]:
metis.observe(src_02, update=True)
hdul_02 = metis.readout(detector_readout_mode="auto")[0]

In [ ]:
metis.observe(sky, update=True)
hdul_sky = metis.readout(detector_readout_mode="auto")[0]

The difference between the background-subtracted 2D-spectra of the two sources is significant. Compare to the images in the first notebook, `LSS_AGN-01_preparation.ipynb`.

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(121)
plt.imshow(hdul_01[1].data - hdul_sky[1].data + 1, origin='lower', norm="log")
plt.subplot(122)
plt.imshow(hdul_02[1].data - hdul_sky[1].data + 1, origin='lower', norm="log")

Save the simulation results in FITS files for analysis with external tools.

In [ ]:
hdul_01.writeto("AGN_prepared-scopesim_lss_n.fits", overwrite=True)
hdul_02.writeto("AGN_rotated_50-scopesim_lss_n.fits", overwrite=True)
hdul_sky.writeto("AGN_background-scopesim_lss_n.fits", overwrite=True)